句子切割

In [1]:
def _cut(sentence):
    """
    将一段文本切分成多个句子
    :param sentence:
    :return:
    """
    new_sentence = []
    sen = []
    for i in sentence:
        if i in ['。', '！', '？', '?'] and len(sen) != 0:
            sen.append(i)
            new_sentence.append("".join(sen))
            sen = []
            continue
        sen.append(i)

    if len(new_sentence) <= 1: # 一句话超过max_seq_length且没有句号的，用","分割，再长的不考虑了。
        new_sentence = []
        sen = []
        for i in sentence:
            if i.split(' ')[0] in ['，', ','] and len(sen) != 0:
                sen.append(i)
                new_sentence.append("".join(sen))
                sen = []
                continue
            sen.append(i)
    if len(sen) > 0:  # 若最后一句话无结尾标点，则加入这句话
        new_sentence.append("".join(sen))
    return new_sentence

In [2]:
def cut_test_set(text_list,len_treshold):
    cut_text_list = []
    cut_index_list = []
    for text in text_list:

        temp_cut_text_list = []
        text_agg = ''
        if len(text) < len_treshold:
            temp_cut_text_list.append(text)
        else:
            sentence_list = _cut(text)  # 一条数据被切分成多句话
            for sentence in sentence_list:
                if len(text_agg) + len(sentence) < len_treshold:
                    text_agg += sentence
                else:
                    temp_cut_text_list.append(text_agg)
                    text_agg = sentence
            temp_cut_text_list.append(text_agg)  # 加上最后一个句子

        cut_index_list.append(len(temp_cut_text_list))
        cut_text_list += temp_cut_text_list

    return cut_text_list, cut_index_list

数据处理

In [3]:
import codecs
import pandas as pd
# df = pd.read_csv('./data/train/label/1697.csv')
# entitys,clss,start_indexs,end_indexs= df['Privacy'],df['Category'],df['Pos_b'],df['Pos_e']
# for entity,cls,start_index,end_index in zip(entitys,clss,start_indexs,end_indexs):
#     length = end_index - start_index + 1
#     print(entity,cls,start_index,end_index,length,len(entity))

# with codecs.open('./data/train/label/1697.csv', "r", encoding="utf-8") as f:
#     lines = f.readlines()
# post_lines = []
# for i,line in enumerate(lines):
#     print(len(line))
#     if len(line) == 2:
#         a = post_lines[-1].split(',')[-1]+line
#         post_lines.pop(-1).append(a)
#         print(post_lines[-1].split(',')[-1]+line)
#     else:
#         post_lines.append(line)
# print(post_lines)
# print(lines)
# for line in lines:
#     line = line.strip('')
#     print(line)
# for i,line in enumerate(lines):
#     print(line)
#     if i == 0:
#         continue
#     print(line)
#     line = line.strip()
#     line_arr = line.split(',')
#     entity = line_arr[-1]
#     cls = line_arr[1]
#     start_index = int(line_arr[2])
#     end_index = int(line_arr[3])
#     length = end_index - start_index + 1
#     print(entity,cls,start_index,end_index,length)
# s = '艺术是相同的，音乐美术体育三样都是艺术。，三样艺术都是靠感觉的。感觉好玩起来就很轻松，所以叫做玩艺术。没感觉找不到北的干脆别玩了！，'
# print(len(s))
# df['Privacy'][2]
        

In [4]:
#设置样本长度
text_length = 250
def from_ann2dic(r_label_path, r_txt_path, w_path, w_file):
    q_dic = {}
#     if r_label_path == './data/train/label/1697.csv':
#         with codecs.open(r_label_path, "r", encoding="utf-8") as f:
#     with codecs.open(r_label_path, "r", encoding="utf-8") as f:
#         lines = f.readlines()
#     i = 0
#     for line in lines:
#         if i == 0:
#             i+=1
#             continue
#         line = line.strip()
#         line_arr = line.split(',')
#         entity = line_arr[-1]
#         print(r_label_path)
#         print(line_arr)
#         cls = line_arr[1]
#         start_index = int(line_arr[2])
#         end_index = int(line_arr[3])
#         length = end_index - start_index + 1
    df = pd.read_csv(r_label_path)
    entitys,clss,start_indexs,end_indexs= df['Privacy'],df['Category'],df['Pos_b'],df['Pos_e']
    for entity,cls,start_index,end_index in zip(entitys,clss,start_indexs,end_indexs):
        length = end_index - start_index + 1
        for r in range(length):
            if r == 0:
                q_dic[start_index] = ("B-%s" % cls)
            else:
                q_dic[start_index + r] = ("I-%s" % cls)

    with codecs.open(r_txt_path, "r", encoding="utf-8") as f:
        content_str = f.read()
        
    
    cut_text_list, cut_index_list = cut_test_set([content_str],text_length)
    
    i = 0
    for idx, line in enumerate(cut_text_list):
        w_path_ = "%s/%s-%s-new.txt" % (w_path, w_file,idx)
        with codecs.open(w_path_, "w", encoding="utf-8") as w:
            for str_ in line:
                if str_ is " " or str_ == "" or str_ == "\n" or str_ == "\r":
                    pass
                else:
                    if i in q_dic:
                        tag = q_dic[i]
                    else:
                        tag = "O"  # 大写字母O
                    w.write('%s %s\n' % (str_, tag))
                i+=1
            w.write('%s\n' % "END O")        

划分数据集

In [5]:
import glob
file_list = glob.glob('./data/train/data/*.txt') 

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train_filelist, val_filelist = train_test_split(file_list,test_size=0.2,random_state=2020)

In [8]:
len(train_filelist)

2012

In [9]:
len(val_filelist)

503

In [10]:
import os
import codecs
# os.mkdir('./data/train_new/')
# os.mkdir('./data/val_new/')

<font color=black size=5 face=雅黑>**训练集处理**</font>

In [11]:
# import os
# data_dir = './data/train/label/'
# r_label_path = os.path.join(data_dir,'1.csv')
# print(r_label_path)

In [12]:
train_filelist

['./data/train/data/408.txt',
 './data/train/data/798.txt',
 './data/train/data/285.txt',
 './data/train/data/950.txt',
 './data/train/data/2011.txt',
 './data/train/data/1106.txt',
 './data/train/data/642.txt',
 './data/train/data/657.txt',
 './data/train/data/879.txt',
 './data/train/data/2193.txt',
 './data/train/data/909.txt',
 './data/train/data/294.txt',
 './data/train/data/1077.txt',
 './data/train/data/1973.txt',
 './data/train/data/7.txt',
 './data/train/data/1338.txt',
 './data/train/data/2139.txt',
 './data/train/data/1070.txt',
 './data/train/data/638.txt',
 './data/train/data/1029.txt',
 './data/train/data/126.txt',
 './data/train/data/456.txt',
 './data/train/data/742.txt',
 './data/train/data/225.txt',
 './data/train/data/1598.txt',
 './data/train/data/69.txt',
 './data/train/data/678.txt',
 './data/train/data/1895.txt',
 './data/train/data/1528.txt',
 './data/train/data/1282.txt',
 './data/train/data/811.txt',
 './data/train/data/2368.txt',
 './data/train/data/549.txt',

In [13]:
data_dir = './data/train/data/'
label_dir = './data/train/label/'
for file in train_filelist:
    if file.find(".txt") == -1:
        continue
    file_name = file.split('/')[-1].split('.')[0]
    print(file_name)
    r_label_path = os.path.join(label_dir, "%s.csv" % file_name)
    r_txt_path = os.path.join(data_dir, "%s.txt" % file_name)
    w_path = './data/train_new/'
    w_file = file_name
    from_ann2dic(r_label_path, r_txt_path, w_path,w_file)

408
798
285
950
2011
1106
642
657
879
2193
909
294
1077
1973
7
1338
2139
1070
638
1029
126
456
742
225
1598
69
678
1895
1528
1282
811
2368
549
121
1359
2512
1463
610
2377
139
1129
1940
887
945
2070
275
1433
1594
482
1355
1059
99
688
2026
2195
451
1073
1203
321
2090
2212
1641
2109
2179
1153
2442
1688
1260
1353
2152
654
1655
1630
2443
327
2233
1816
2057
1346
1296
2028
2429
1190
606
1849
1596
2076
691
1045
32
2441
1184
876
1017
88
1135
2061
918
54
2181
880
1014
630
1469
1924
91
804
1500
42
2445
1390
2367
1247
200
1730
1341
1617
1013
1188
590
546
292
324
202
1504
190
1566
1307
2384
79
1368
296
2260
137
2183
485
2395
140
1863
1407
84
421
769
377
2434
2082
1768
514
473
2329
1757
2137
432
566
323
364
396
2401
1213
1836
542
1068
1745
2006
1078
279
2127
780
145
1308
2308
1040
946
970
745
2514
1702
1712
1851
1743
96
755
62
178
2213
400
270
947
497
1217
2153
2449
2180
1186
1094
269
861
665
808
1945
1625
1456
1371
1144
1786
1419
1737
1676
2374
2436
1321
2405
253
962
2425
1527
205
2166
1383
1540
71

2228
2262
2370
2454
584
2347
2330
1541
1176
1957
1179
771
2324
2414
870
516
2300
1397
1035
577
406
569
94
1520
1238
2243
543
1837
1281
1990
338
550
1767
1807
1221
579
1586
176
844
681
710
572
1723
2083
82
1662
2484
1225
469
201
1976
177
528
803
1931
2167
559
1437
2027
2469
272
1918
1373
1187
2502
2294
1084
553
203
1324
2346
318
2165
1312
626
747
1765
89
1272
763
1127
1450
1231
12
659
1316
1102
1227
851
501
1336
47
835
692
1423
969
1003
479
2190
955
397
186
1431
1812
437
2340
2000
1756
2047
1874
919
875
505
196
2386
1412
2067
2440
1328
1126
492
170
348
1742
611
278
1609
480
1603
2087
1703
1410
1047
1386
2503
2114
1597
1019
2140
1828
368
13
1286
1992
239
1317
307
1262
1587
360
1425
779
1639
608
1749
1979
652
1302
464
243
2399
1673
2500
1132
267
1695
1023
413
871
65
539
206
2211
1781
1173
1561
628
1122
162
356
53
1826
2063
1501
1139
513
2493
660
1350
1776


<font color=black size=5 face=雅黑>**验证集处理**</font>

In [14]:
# <font color=black size=5 face=雅黑>**Hello world**</font>

In [15]:
data_dir = './data/train/data/'
label_dir = './data/train/label/'
for file in val_filelist:
    if file.find(".txt") == -1:
        continue
    file_name = file.split('/')[-1].split('.')[0]
    r_csv_path = os.path.join(label_dir, "%s.csv" % file_name)
    r_txt_path = os.path.join(data_dir, "%s.txt" % file_name)
    w_path = './data/val_new/'
    w_file = file_name
    from_ann2dic(r_csv_path, r_txt_path, w_path,w_file)

<font color=black size=5 face=雅黑>**训练集合并**</font>

In [16]:
w_path = "./data/train.txt"
for file in os.listdir('./data/train_new/'):
    path = os.path.join("./data/train_new", file)
    if not file.endswith(".txt"):  
        continue
    q_list = []
    print("开始读取文件:%s" % file)
    with codecs.open(path, "r", encoding="utf-8") as f:
        line = f.readline()
        line = line.strip("\n\r")
        while line != "END O":
            q_list.append(line)
            line = f.readline()
            line = line.strip("\n\r")
    print("开始写入文本%s" % w_path)
    with codecs.open(w_path, "a", encoding="utf-8") as f:
        for item in q_list:
            if item.__contains__('\ufeff1'):
                print("===============")
            f.write('%s\n' % item)
        f.write('\n')
    f.close()

开始读取文件:408-0-new.txt
开始写入文本./data/train.txt
开始读取文件:798-0-new.txt
开始写入文本./data/train.txt
开始读取文件:285-0-new.txt
开始写入文本./data/train.txt
开始读取文件:950-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2011-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1106-0-new.txt
开始写入文本./data/train.txt
开始读取文件:642-0-new.txt
开始写入文本./data/train.txt
开始读取文件:657-0-new.txt
开始写入文本./data/train.txt
开始读取文件:879-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2193-0-new.txt
开始写入文本./data/train.txt
开始读取文件:909-0-new.txt
开始写入文本./data/train.txt
开始读取文件:909-1-new.txt
开始写入文本./data/train.txt
开始读取文件:294-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1077-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1973-0-new.txt
开始写入文本./data/train.txt
开始读取文件:7-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1338-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2139-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1070-0-new.txt
开始写入文本./data/train.txt
开始读取文件:638-0-new.txt
开始写入文本./data/train.txt
开始读取文件:638-1-new.txt
开始写入文本./data/train.txt
开始读取文件:638-2-new.txt
开始写入文本./data/train.txt
开始读取文件:1029-0-new.txt
开始写入

开始写入文本./data/train.txt
开始读取文件:62-0-new.txt
开始写入文本./data/train.txt
开始读取文件:178-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2213-0-new.txt
开始写入文本./data/train.txt
开始读取文件:400-0-new.txt
开始写入文本./data/train.txt
开始读取文件:270-0-new.txt
开始写入文本./data/train.txt
开始读取文件:947-0-new.txt
开始写入文本./data/train.txt
开始读取文件:497-0-new.txt
开始写入文本./data/train.txt
开始读取文件:497-1-new.txt
开始写入文本./data/train.txt
开始读取文件:1217-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2153-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2449-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2180-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1186-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1094-0-new.txt
开始写入文本./data/train.txt
开始读取文件:269-0-new.txt
开始写入文本./data/train.txt
开始读取文件:861-0-new.txt
开始写入文本./data/train.txt
开始读取文件:861-1-new.txt
开始写入文本./data/train.txt
开始读取文件:665-0-new.txt
开始写入文本./data/train.txt
开始读取文件:808-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1945-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1625-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1456-0-new.txt
开始写入文本./data/train.txt


开始写入文本./data/train.txt
开始读取文件:2499-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1687-0-new.txt
开始写入文本./data/train.txt
开始读取文件:472-0-new.txt
开始写入文本./data/train.txt
开始读取文件:958-0-new.txt
开始写入文本./data/train.txt
开始读取文件:797-0-new.txt
开始写入文本./data/train.txt
开始读取文件:561-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1252-0-new.txt
开始写入文本./data/train.txt
开始读取文件:175-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1659-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2145-0-new.txt
开始写入文本./data/train.txt
开始读取文件:255-0-new.txt
开始写入文本./data/train.txt
开始读取文件:951-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2282-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2482-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1920-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1920-1-new.txt
开始写入文本./data/train.txt
开始读取文件:16-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2216-0-new.txt
开始写入文本./data/train.txt
开始读取文件:214-0-new.txt
开始写入文本./data/train.txt
开始读取文件:306-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2239-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2239-1-new.txt
开始写入文本./data/train.tx

开始读取文件:640-0-new.txt
开始写入文本./data/train.txt
开始读取文件:381-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1923-0-new.txt
开始写入文本./data/train.txt
开始读取文件:903-0-new.txt
开始写入文本./data/train.txt
开始读取文件:435-0-new.txt
开始写入文本./data/train.txt
开始读取文件:435-1-new.txt
开始写入文本./data/train.txt
开始读取文件:979-0-new.txt
开始写入文本./data/train.txt
开始读取文件:345-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1815-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1101-0-new.txt
开始写入文本./data/train.txt
开始读取文件:650-0-new.txt
开始写入文本./data/train.txt
开始读取文件:650-1-new.txt
开始写入文本./data/train.txt
开始读取文件:650-2-new.txt
开始写入文本./data/train.txt
开始读取文件:1264-0-new.txt
开始写入文本./data/train.txt
开始读取文件:470-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1299-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1216-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1216-1-new.txt
开始写入文本./data/train.txt
开始读取文件:259-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1370-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1370-1-new.txt
开始写入文本./data/train.txt
开始读取文件:1677-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1653-0-new.txt


开始写入文本./data/train.txt
开始读取文件:2458-0-new.txt
开始写入文本./data/train.txt
开始读取文件:619-0-new.txt
开始写入文本./data/train.txt
开始读取文件:619-1-new.txt
开始写入文本./data/train.txt
开始读取文件:213-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1822-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2398-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1292-0-new.txt
开始写入文本./data/train.txt
开始读取文件:867-0-new.txt
开始写入文本./data/train.txt
开始读取文件:614-0-new.txt
开始写入文本./data/train.txt
开始读取文件:184-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2356-0-new.txt
开始写入文本./data/train.txt
开始读取文件:635-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1971-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1313-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1313-1-new.txt
开始写入文本./data/train.txt
开始读取文件:1313-2-new.txt
开始写入文本./data/train.txt
开始读取文件:1313-3-new.txt
开始写入文本./data/train.txt
开始读取文件:1725-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1725-1-new.txt
开始写入文本./data/train.txt
开始读取文件:1725-2-new.txt
开始写入文本./data/train.txt
开始读取文件:819-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1922-0-new.txt
开始写入文本./data/train

...

开始读取文件:2065-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2065-1-new.txt
开始写入文本./data/train.txt
开始读取文件:1266-0-new.txt
开始写入文本./data/train.txt
开始读取文件:301-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1234-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1031-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1384-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1384-1-new.txt
开始写入文本./data/train.txt
开始读取文件:1384-2-new.txt
开始写入文本./data/train.txt
开始读取文件:313-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1888-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1793-0-new.txt
开始写入文本./data/train.txt
开始读取文件:487-0-new.txt
开始写入文本./data/train.txt
开始读取文件:130-0-new.txt
开始写入文本./data/train.txt
开始读取文件:799-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1175-0-new.txt
开始写入文本./data/train.txt
开始读取文件:773-0-new.txt
开始写入文本./data/train.txt
开始读取文件:519-0-new.txt
开始写入文本./data/train.txt
开始读取文件:519-1-new.txt
开始写入文本./data/train.txt
开始读取文件:675-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1621-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2012-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1779-0-new.t

开始写入文本./data/train.txt
开始读取文件:1827-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2408-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1253-0-new.txt
开始写入文本./data/train.txt
开始读取文件:841-0-new.txt
开始写入文本./data/train.txt
开始读取文件:510-0-new.txt
开始写入文本./data/train.txt
开始读取文件:510-1-new.txt
开始写入文本./data/train.txt
开始读取文件:2481-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2390-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2390-1-new.txt
开始写入文本./data/train.txt
开始读取文件:2390-2-new.txt
开始写入文本./data/train.txt
开始读取文件:1115-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1521-0-new.txt
开始写入文本./data/train.txt
开始读取文件:591-0-new.txt
开始写入文本./data/train.txt
开始读取文件:683-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2210-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2210-1-new.txt
开始写入文本./data/train.txt
开始读取文件:1563-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2406-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2373-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2373-1-new.txt
开始写入文本./data/train.txt
开始读取文件:1808-0-new.txt
开始写入文本./data/train.txt
开始读取文件:906-0-new.txt
开始写入文本./data/tra

开始写入文本./data/train.txt
开始读取文件:1149-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1460-0-new.txt
开始写入文本./data/train.txt
开始读取文件:475-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2130-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2130-1-new.txt
开始写入文本./data/train.txt
开始读取文件:2378-0-new.txt
开始写入文本./data/train.txt
开始读取文件:593-0-new.txt
开始写入文本./data/train.txt
开始读取文件:146-0-new.txt
开始写入文本./data/train.txt
开始读取文件:34-0-new.txt
开始写入文本./data/train.txt
开始读取文件:740-0-new.txt
开始写入文本./data/train.txt
开始读取文件:49-0-new.txt
开始写入文本./data/train.txt
开始读取文件:49-1-new.txt
开始写入文本./data/train.txt
开始读取文件:49-2-new.txt
开始写入文本./data/train.txt
开始读取文件:49-3-new.txt
开始写入文本./data/train.txt
开始读取文件:49-4-new.txt
开始写入文本./data/train.txt
开始读取文件:988-0-new.txt
开始写入文本./data/train.txt
开始读取文件:684-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1067-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2397-0-new.txt
开始写入文本./data/train.txt
开始读取文件:976-0-new.txt
开始写入文本./data/train.txt
开始读取文件:666-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1432-0-new.txt
开始写入文本./data/train.txt
开始读取文件:

开始写入文本./data/train.txt
开始读取文件:734-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1325-0-new.txt
开始写入文本./data/train.txt
开始读取文件:156-0-new.txt
开始写入文本./data/train.txt
开始读取文件:562-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1445-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1445-1-new.txt
开始写入文本./data/train.txt
开始读取文件:548-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1215-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1271-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2001-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2361-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2394-0-new.txt
开始写入文本./data/train.txt
开始读取文件:810-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1147-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1858-0-new.txt
开始写入文本./data/train.txt
开始读取文件:392-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1082-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1158-0-new.txt
开始写入文本./data/train.txt
开始读取文件:189-0-new.txt
开始写入文本./data/train.txt
开始读取文件:245-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1799-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2359-0-new.txt
开始写入文本./data/train

开始读取文件:348-1-new.txt
开始写入文本./data/train.txt
开始读取文件:1742-0-new.txt
开始写入文本./data/train.txt
开始读取文件:611-0-new.txt
开始写入文本./data/train.txt
开始读取文件:278-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1609-0-new.txt
开始写入文本./data/train.txt
开始读取文件:480-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1603-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2087-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1703-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1410-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1410-1-new.txt
开始写入文本./data/train.txt
开始读取文件:1047-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1386-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1386-1-new.txt
开始写入文本./data/train.txt
开始读取文件:2503-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2114-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2114-1-new.txt
开始写入文本./data/train.txt
开始读取文件:1597-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1597-1-new.txt
开始写入文本./data/train.txt
开始读取文件:1019-0-new.txt
开始写入文本./data/train.txt
开始读取文件:2140-0-new.txt
开始写入文本./data/train.txt
开始读取文件:1828-0-new.txt
开始写入文本./data/train.txt
开始读取文件:368-0-n

<font color=black size=5 face=雅黑>**验证集合并**</font>

In [17]:
w_path = "./data/val.txt"
for file in os.listdir('./data/val_new/'):
    path = os.path.join("./data/val_new", file)
    if not file.endswith(".txt"):  
        continue
    q_list = []

    with codecs.open(path, "r", encoding="utf-8") as f:
        line = f.readline()
        line = line.strip("\n\r")
        while line != "END O":
            q_list.append(line)
            line = f.readline()
            line = line.strip("\n\r")
    
    with codecs.open(w_path, "a", encoding="utf-8") as f:
        for item in q_list:
            if item.__contains__('\ufeff1'):
                print("===============")
            f.write('%s\n' % item)
        f.write('\n')
    f.close()

In [18]:
for file in val_filelist:
    file_name = file.split('/')[-1].split('.')[0]
    r_ann_path = os.path.join("./data/train/label/", "%s.csv" % file_name)
    os.system("cp %s %s"%(file,"./data/val_data"))
    os.system("cp %s %s"%(r_ann_path,"./data/val_data"))
    print(file)

./data/train/data/2403.txt
./data/train/data/1846.txt
./data/train/data/25.txt
./data/train/data/2313.txt
./data/train/data/964.txt
./data/train/data/1824.txt
./data/train/data/319.txt
./data/train/data/2249.txt
./data/train/data/585.txt
./data/train/data/2096.txt
./data/train/data/2230.txt
./data/train/data/899.txt
./data/train/data/1516.txt
./data/train/data/101.txt
./data/train/data/1624.txt
./data/train/data/1393.txt
./data/train/data/2102.txt
./data/train/data/1556.txt
./data/train/data/2135.txt
./data/train/data/223.txt
./data/train/data/373.txt
./data/train/data/1109.txt
./data/train/data/981.txt
./data/train/data/1985.txt
./data/train/data/1718.txt
./data/train/data/694.txt
./data/train/data/298.txt
./data/train/data/2450.txt
./data/train/data/2422.txt
./data/train/data/733.txt
./data/train/data/1113.txt
./data/train/data/1354.txt
./data/train/data/2290.txt
./data/train/data/757.txt
./data/train/data/1347.txt
./data/train/data/1145.txt
./data/train/data/762.txt
./data/train/dat

./data/train/data/1048.txt
./data/train/data/1847.txt
./data/train/data/855.txt
./data/train/data/2048.txt
./data/train/data/2.txt
./data/train/data/2177.txt
./data/train/data/1529.txt
./data/train/data/1869.txt
./data/train/data/1760.txt
./data/train/data/404.txt
./data/train/data/534.txt
./data/train/data/2225.txt
./data/train/data/55.txt
./data/train/data/466.txt
./data/train/data/1991.txt
./data/train/data/171.txt
./data/train/data/1004.txt
./data/train/data/2364.txt
./data/train/data/2301.txt
./data/train/data/1902.txt
./data/train/data/1161.txt
./data/train/data/842.txt
./data/train/data/2005.txt
./data/train/data/1870.txt
./data/train/data/1455.txt
./data/train/data/2160.txt
./data/train/data/2410.txt
./data/train/data/1544.txt
./data/train/data/2052.txt
./data/train/data/1700.txt
./data/train/data/9.txt
./data/train/data/724.txt
./data/train/data/412.txt
./data/train/data/1010.txt
./data/train/data/1513.txt
./data/train/data/1193.txt
./data/train/data/1898.txt
./data/train/data